In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents import create_sql_agent, load_tools
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.tools import HumanInputRun

from langchain.agents import AgentType, Tool, initialize_agent

from getpass import getpass
import os

In [2]:
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

In [22]:
csv_agent = create_csv_agent(
    llm,
    "titanic.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [23]:
db = SQLDatabase.from_uri("sqlite:////home/suchitg/agent_blog/Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [71]:
tools = [
    Tool(
        name="CSV data interaction and QA agent",
        func=csv_agent.run,
        description="useful when you need to work with csv files. Input should be a fully formed question."
    ),
    Tool(
        name="SQL database interaction and QA agent",
        func=sql_agent.run,
        description="useful when you need to work with sql databases. Input should be a fully formed question."
    ),
    HumanInputRun(description="You can ask a human for guidance when you are not sure what to do next or want to know the type of data to query. The input should be a question for the human.")
]

In [80]:
data_analyst_agent = initialize_agent(
    tools, llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
    agent_kwargs={"prefix": """Answer the following questions as best you can. Always clarify the filetype to query using the human tool first. You have access to the following tools:"""}
)

In [81]:
print(data_analyst_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. Always clarify the filetype to query using the human tool first. You have access to the following tools:

CSV data interaction and QA agent: useful when you need to work with csv files. Input should be a fully formed question.
SQL database interaction and QA agent: useful when you need to work with sql databases. Input should be a fully formed question.
human: You can ask a human for guidance when you think you got stuck or you are not sure what to do next or want to know the type of data to query. The input should be a question for the human.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [CSV data interaction and QA agent, SQL database interaction and QA agent, human]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Though

In [82]:
data_analyst_agent.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
I need to find information about the playlisttrack table, which is likely stored in a database.
Action: SQL database interaction and QA agent
Action Input: "Can you provide me with the schema for the playlisttrack table?"

> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I can now query the schema for the playlisttrack table.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I now know the final answer
Final Answer: The schema for the Pla

'The PlaylistTrack table has two columns: PlaylistId of type INTEGER and TrackId of type INTEGER.'

In [83]:
data_analyst_agent.run("How many people have more than 3 siblings?")



> Entering new AgentExecutor chain...
I need to query a database to find the number of people with more than 3 siblings.
Action: human
Action Input: How is the data on people and their number of siblings stored?

How is the data on people and their number of siblings stored?

Observation: its a csv file
Thought:I should use the CSV data interaction and QA agent to query the csv file.
Action: CSV data interaction and QA agent
Action Input: SELECT COUNT(*) FROM people WHERE siblings > 3

> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the SibSp column is greater than 3, and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].shape[0]
Observation: 30
Thought:I now know the final answer
Final Answer: 30

> Finished chain.

Observation: 30
Thought:I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

In [84]:
data_analyst_agent.run("How many categorical and continous variables are there in the titanic dataset?")



> Entering new AgentExecutor chain...
I need to find out the type of data to query in the titanic dataset.
Action: human
Action Input: What type of data is the titanic dataset?

What type of data is the titanic dataset?

Observation: csv
Thought:I need to use the CSV data interaction and QA agent to answer this question.
Action: CSV data interaction and QA agent
Action Input: How many categorical and continuous variables are there in the titanic dataset?

> Entering new AgentExecutor chain...
Thought: I can count the unique values in each column to determine the categorical variables, and check the data types to determine the continuous variables.
Action: python_repl_ast
Action Input: df.nunique()
Observation: PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64
Thought:There are 5 categorical varia

'There are 5 categorical variables and 6 continuous variables in the titanic dataset.'